主成分分析（Principal Component Analysis，PCA）是一种多变量统计方法，它是最常用的降维方法之一，通过正交变换将一组可能存在相关性的变量数据转换为一组线性不相关的变量，转换后的变量被称为主成分。

可以使用两种方法进行 PCA，分别是特征分解或奇异值分解（SVD）。

# 先从协方差说起

协方差$$cov(X,Y)=\frac{\sum_{i=1}^n{(X_i-\bar{X})(Y_i-\bar{Y})}}{n-1}$$

协方差其意义：
度量各个维度偏离其均值的程度。协方差的值如果为正值，则说明两者是正相关的(从协方差可以引出“相关系数”的定义)，结果为负值就说明负相关的，如果为0，也是就是统计上说的“相互独立”。



协方差矩阵：  
$$C = \begin{pmatrix}
    cov(x,x) &cov(x,y)  & cov(x,z) \\
   cov(y,x)  & cov(y,y) &  cov(y,z) \\
   cov(z,x)  & cov(z,y) &cov(z,z)   \\
\end{pmatrix}\qquad$$  
从协方差矩阵上，可以得到变量之间两两的相关性

In [ ]:
%pip install mpld3
#%pip install torch

In [ ]:
import numpy as np
#设置一下np的输出格式
np.set_printoptions(threshold=100,precision= 4,suppress=True)


In [ ]:
# 计算以下数据的协方差矩阵
import numpy as np
np.random.seed(0)
data = np.random.uniform(1,10,(10,2))
data[:,1:] = 0.5*data[:,0:1]+np.random.uniform(-2,2,(10,1))
data



必须去中心化

In [ ]:
data_norm = data-data.mean(axis = 0)
data_norm

In [ ]:
X = data_norm[:,0]
Y = data_norm[:,1]
X,Y

In [ ]:
# 定义一个函数，输入X，Y能得到X，Y之间的协方差
def getcov(X,Y):

    covxy = ((X-X.mean())*(Y-Y.mean())).sum()/(len(X)-1)

    return covxy

In [ ]:
getcov(X,X)

numpy自带了协方差矩阵的计算方法，验证一下

In [ ]:
C = np.cov(data_norm.T)
C

# 计算协方差矩阵的特征向量和特征值

由矩阵特征值特征向量的定义：
$$Cv=\lambda v$$
其中，$λ$是特征向量$v$对应的特征值，一个矩阵的一组特征向量是一组正交向量。

特征值分解矩阵

对于矩阵$C$，有一组特征向量$V$，将这组向量进行正交化单位化，就能得到一组正交单位向量。特征值分解，就是将矩阵$C$分解为如下式：

$$C=Q\Sigma Q^{-1}$$

其中，$Q$是矩阵$C$的特征向量组成的矩阵，$\Sigma$则是一个对角阵，对角线上的元素就是特征值。

In [ ]:
#计算特征值和特征向量
vals, vecs = np.linalg.eig(C)
#重新排序，从大到小
vecs = vecs[:,np.argsort(-vals)]
vals = vals[np.argsort(-vals)]


In [ ]:
#第一个特征值对应的特征向量
vals[0],vecs[:,0]

In [ ]:
#第二个特征值对应的特征向量
vals[1],vecs[:,1]

这时候，相当于已经在数据中定义了两个轴，第一个轴的方向是第一个特征向量$v_1$，第二个轴的方向是第二个特征向量$v_2$


In [ ]:
import matplotlib.pyplot as plt
#设置图大小
size = 15

plt.figure(1,(8,8))

plt.scatter(data[:,0],data[:,1],label='origin data')

i=0
ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
ev = (ev+data.mean(0))
plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

i=1
ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
ev = (ev+data.mean(0))
plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

#plt.plot(vecs[:,1]*-10,vecs[:,1]*10)

#画一下x轴y轴
plt.plot([-size,size],[0,0],c='black')
plt.plot([0,0],[-size,size],c='black')
plt.xlim(-size,size)
plt.ylim(-size,size)
plt.legend()
plt.show()

如果用PCA把$m$个维度的数据降维成$k$个维度，即只用前$k$个主成分来表示，那么数据在主成分上的投影坐标是
$$Y_{n*k} = X_{n*m}Q_{m*k}$$
$Q$为特征向量组成的矩阵

In [ ]:
#数据在主成分1上的投影坐标是Y
k=1
Q = vecs[:,:k]
Y = np.matmul(data_norm,Q)
Y

这个时候我们相当于只需要存储**前k个主成分的特征向量$Q_{m*k}$**和**数据在前k个主成分上的投影坐标$Y_{n*k}$**，就可以还原数据
$$Y_{n*k}{Q_{m*k}}^T=X_{n*m}$$
其中，由于$Q$已经正交化，${Q_{m*k}}^T{Q_{m*k}}=I_{k*k}$

In [ ]:
#得到去中心化的还原数据
np.matmul(Y,Q.T)

In [ ]:
#加上均值，还原数据
data_ = np.matmul(Y,Q.T)+data.mean(0)
data_

# 降维重构的数据与原数据对比

## 用我们刚刚的方法

In [ ]:

import matplotlib.pyplot as plt
#设置图大小
size = 10

plt.figure(1,(8,8))

plt.scatter(data[:,0],data[:,1],label='origin data')

plt.scatter(data_[:,0],data_[:,1],label='restructured data')

i=0
ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
ev = (ev+data.mean(0))
plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

i=1
ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
ev = (ev+data.mean(0))
plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

#plt.plot(vecs[:,1]*-10,vecs[:,1]*10)

#画一下x轴y轴
plt.plot([-size,size],[0,0],c='black')
plt.plot([0,0],[-size,size],c='black')
plt.xlim(-size,size)
plt.ylim(-size,size)
plt.legend()
plt.show()

## 用sklearn的PCA

In [ ]:

import matplotlib.pyplot as plt

#sklearn PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2) 
pca.fit(data)
Y = pca.fit_transform(data)
vecs = pca.components_.T
data_ = np.dot(Y[:,:1],vecs[:,:1].T)+data.mean(0)


#设置图大小
size = 10

plt.figure(1,(8,8))

plt.scatter(data[:,0],data[:,1],label='origin data')

plt.scatter(data_[:,0],data_[:,1],label='restructured data')

i=0
ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
ev = (ev+data.mean(0))
plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

i=1
ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
ev = (ev+data.mean(0))
plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

#plt.plot(vecs[:,1]*-10,vecs[:,1]*10)

#画一下x轴y轴
plt.plot([-size,size],[0,0],c='black')
plt.plot([0,0],[-size,size],c='black')
plt.xlim(-size,size)
plt.ylim(-size,size)
plt.legend()
plt.show()


## 用SVD

In [ ]:
#用SVD主要是想用SVD求出主成分的方向向量
U,vals,V = np.linalg.svd(data_norm)
vecs = V.T

#数据在主成分1上的投影坐标是Y
Y = np.matmul(data_norm,vecs[:,:1])
#得到去中心化的还原数据
data_ = np.matmul(Y,vecs[:,:1].T)+data.mean(0)


In [ ]:
#画图
import matplotlib.pyplot as plt
#设置图大小
size = 10

plt.figure(1,(8,8))

plt.scatter(data[:,0],data[:,1],label='origin data')

plt.scatter(data_[:,0],data_[:,1],label='restructured data')

#绘制主成分的向量
i=0
ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
ev = (ev+data.mean(0))
plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

i=1
ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
ev = (ev+data.mean(0))
plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

#plt.plot(vecs[:,1]*-10,vecs[:,1]*10)

#画一下x轴y轴
plt.plot([-size,size],[0,0],c='black')
plt.plot([0,0],[-size,size],c='black')
plt.xlim(-size,size)
plt.ylim(-size,size)
plt.legend()
plt.show()

# 三维数据的测试

In [ ]:
# 计算以下数据的协方差矩阵
import numpy as np
np.random.seed(0)
data = np.random.uniform(-10,10,(20,3))
data[:,1:] = 0.5*data[:,0:1]+np.random.uniform(-2,2,(20,1))
data[:,2:] = 0.5*data[:,0:1]+np.random.uniform(-3,3,(20,1))


data_normal = data-data.mean(0)


In [ ]:
C = np.cov(data_normal.T)

#计算特征值和特征向量
vals, vecs = np.linalg.eig(C)
#重新排序，从大到小
vecs = vecs[:,np.argsort(-vals)]
vals = vals[np.argsort(-vals)]

vals,vecs

In [ ]:
import matplotlib.pyplot as plt, mpld3
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm


X = data[:,0]
Y = data[:,1]
Z = data[:,2]

for i in range(45,60):
    #让图转起来
    import IPython
    IPython.display.clear_output(wait=True)

    fig = plt.figure(1,(6,4),dpi = 250)
    ax = fig.gca(projection='3d')

    plt.cla()
    #绘制散点
    ax.scatter(X,Y,Z,s=5)

    #绘制xyz轴
    ax.plot([0,0],[0,0],[-10,10],c = 'black',linewidth = 0.8)
    ax.plot([0,0],[-10,10],[0,0],c = 'black',linewidth = 0.8)
    ax.plot([-10,10],[0,0],[0,0],c = 'black',linewidth = 0.8)
    
    
    ax.view_init(azim=i)
    plt.xlim(-X.max(), X.max())
    plt.ylim(-Y.max(), Y.max())
    ax.set_zlim(-Z.max(),Z.max())
    plt.title('PCA')
    plt.show()
    

In [ ]:
#数据在主成分1上的投影坐标是
zcf = np.matmul(data_normal,vecs[:,:2])
data_ = np.matmul(zcf,vecs[:,:2].T)+data.mean(0)
data_

In [ ]:
import matplotlib.pyplot as plt, mpld3
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm


X = data[:,0]
Y = data[:,1]
Z = data[:,2]

X_ = data_[:,0]
Y_ = data_[:,1]
Z_ = data_[:,2]

for i in range(35,55):
    #让图转起来
    import IPython
    IPython.display.clear_output(wait=True)

    fig = plt.figure(1,(6,4),dpi = 250)
    ax = fig.gca(projection='3d')

    plt.cla()
    #绘制散点
    ax.scatter(X,Y,Z,s=5,label='origin data')
    
    ax.scatter(X_,Y_,Z_,s=5,label='restructured data')
    ax.plot_trisurf(X_, Y_, Z_)
    #绘制xyz轴
    ax.plot([0,0],[0,0],[-10,10],c = 'black',linewidth = 0.8)
    ax.plot([0,0],[-10,10],[0,0],c = 'black',linewidth = 0.8)
    ax.plot([-10,10],[0,0],[0,0],c = 'black',linewidth = 0.8)
    
    
    ax.view_init(azim=i)
    plt.xlim(-X.max(), X.max())
    plt.ylim(-Y.max(), Y.max())
    ax.set_zlim(-Z.max(),Z.max())
    plt.title('PCA')
    plt.show()
    

# 这关奇异值什么事？

## 奇异值分解  $Singular$ $Value$ $Decomposition$ $(SVD)$

奇异值分解是一个能适用于任意矩阵的一种分解的方法，对于任意矩阵A总是存在一个奇异值分解：

$$A=U\Sigma V^T$$

假设A是一个$m*n$的矩阵，那么得到的$U$是一个$m*m$的方阵，$U$里面的正交向量被称为左奇异向量。  
$Σ$是一个$m*n$的矩阵，$Σ$除了对角线其它元素都为0，对角线上的元素称为奇异值。  
$V^T$是$V$的转置矩阵，是一个$n*n$的矩阵，它里面的正交向量被称为右奇异值向量。  
而且一般来讲，我们会将$Σ$上的值按从大到小的顺序排列。


## SVD分解矩阵A的步骤：

1) 求的$AA^T$特征值和特征向量，用单位化的特征向量构成$U$。  
2) 求的$A^TA$特征值和特征向量，用单位化的特征向量构成$V$。  
3) 将$AA^T$或者$A^TA$的特征值求平方根，然后构成$Σ$。

## 我们自己实现

In [ ]:
#求U
valsU,U = np.linalg.eig(np.dot(data_normal,data_normal.T))
#以特征值排序
U = U[:,np.argsort(-valsU)]
valsU = valsU[np.argsort(-valsU)]

#求V
valsV,V = np.linalg.eig(np.dot(data_normal.T,data_normal))
#以特征值排序
V = V[:,np.argsort(-valsV)]
valsV = valsV[np.argsort(-valsV)]
V = V.T

#求对角阵
vals = valsV**0.5
U,vals,V

In [ ]:
plt.imshow(np.real(U))
plt.title('$U$')
plt.show()

m,n = data_normal.shape
sigma = np.zeros((m,n))
for i in range(min([m,n])):
    sigma[i][i]=vals[i]

plt.imshow(sigma)
plt.title('$\Sigma$')
plt.show()

plt.imshow(np.real(V))
plt.title('$V$')
plt.show()

In [ ]:
#重构矩阵
m,n = data_normal.shape
sigma = np.zeros((m,n))
for i in range(min([m,n])):
    sigma[i][i]=vals[i]
np.real(data_normal-np.dot(np.dot(U,sigma),V))

## np自带的svd实现

In [ ]:
U,vals,V = np.linalg.svd(data_normal)
U

In [ ]:
#重构矩阵
m,n = data_normal.shape
sigma = np.zeros((m,n))
for i in range(min([m,n])):
    sigma[i][i]=vals[i]
data_normal-np.dot(np.dot(U,sigma),V)

In [ ]:
plt.imshow(np.real(U))
plt.title('U')
plt.show()

m,n = data_normal.shape
sigma = np.zeros((m,n))
for i in range(min([m,n])):
    sigma[i][i]=vals[i]

plt.imshow(sigma)
plt.title('$\Sigma$')
plt.show()

plt.imshow(np.real(V))
plt.title('V')
plt.show()

In [ ]:
#重构矩阵
data_normal-np.dot(np.dot(U,sigma),V)

注意到：用我们自己的方法做SVD的时候，结果和np的SVD结果不一样，这是因为U矩阵和V矩阵的正负问题，如果两者都取负，则抵消，结果也不变


## 为什么要用SVD来做PCA

**很巧的是，SVD中的右奇异矩阵V，就是PCA的主成分**

在PCA降维中，我们需要找到样本协方差矩阵$C$的最大$k$个特征向量，然后用这最大的$k$个特征向量组成的矩阵来做低维投影降维。  
可以看出，在这个过程中需要先求出协方差矩阵,当样本数多、样本特征数也多的时候，这个计算量还是很大的。



# PCA的缺陷
如果加入异常值，PCA的结果会发生什么变化？

In [ ]:


import numpy as np

for i in range(0,720,10):
    
    np.random.seed(0)
    data = np.random.uniform(1,10,(20,2))
    data[:,1:] = 0.5*data[:,0:1]+np.random.uniform(-2,2,(20,1))
    


    #如果我们在数据中加入异常值,定义一个角度放入异常值
    i = 2*i/360*np.pi
    data_outsider=[np.sin(i)*5*i/(2*np.pi),np.cos(i)*5*i/(2*np.pi)]
    data = np.vstack((data,data_outsider))
    
    #去中心化
    data_normal = data-data.mean(0)
    X = data_normal[:,0]
    Y = data_normal[:,1]
    
    #协方差矩阵
    C = np.cov(data_normal.T)
    
    #计算特征值和特征向量
    vals, vecs = np.linalg.eig(C)
    
    #重新排序，从大到小
    vecs = vecs[:,np.argsort(-vals)]
    vals = vals[np.argsort(-vals)]

    #数据在主成分1上的投影坐标是
    zcf1 = np.matmul(data_normal,vecs[:,0])
    
    #只用主成分1重构数据
    data_ = np.matmul(zcf1.reshape(len(data),1),vecs[:,0].reshape(1,2))+data.mean(0)

    import matplotlib.pyplot as plt
    #设置图大小
    size = 10
    
    #让图转起来，每次绘图都清空掉上一次绘制结果
    import IPython
    IPython.display.clear_output(wait=True)
          
    plt.figure(1,(8,8))

    plt.scatter(data[:,0],data[:,1],label='origin data')
    plt.scatter(data_[:,0],data_[:,1],label='restructured data')
    plt.scatter(data[-1,0],data[-1,1],label='outsider')

    i=0
    
    #绘制两个主成分的方向
    ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
    ev = (ev+data.mean(0))
    plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

    i=1
    ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
    ev = (ev+data.mean(0))
    plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))
    
    #绘制一下原始数据和重构数据的连线
    for j in range(len(data)):
        plt.plot([data[j,0],data_[j,0]],[data[j,1],data_[j,1]],c='black',linewidth = 0.5)
        

    #画一下x轴y轴
    plt.plot([-size,size],[0,0],c='black')
    plt.plot([0,0],[-size,size],c='black')
    plt.xlim(-12.5,12.5)
    plt.ylim(-12.5,12.5)
    plt.title('result of PCA')
    plt.legend()
    plt.show()

# Stable Principle Component Pursuit (SPCP)
 Stable Principle Component Pursuit (SPCP)是PCA的改进版本，它把问题变成：
$$\min_{L,S}{{ \left\Vert {L} \right\Vert }_*+\lambda{ \left\Vert {S} \right\Vert }_1}$$
$$s.t. M=L+S$$
其中：
$M$为原始矩阵  
$L$为低秩矩阵，${ \left\Vert {L} \right\Vert }_*$为$L$矩阵核范数Nuclear Norm,为矩阵奇异值的和(用于表示低秩矩阵)  
$S$为稀疏矩阵，${ \left\Vert {S} \right\Vert }_1$为矩阵L1范数（列模）:表示矩阵中非零元素的绝对值之和  
$\lambda=1/\sqrt{max(m,n)}$  

即，把原始矩阵分解为低秩矩阵与稀疏矩阵的和，这样子稀疏矩阵代表的就是矩阵的异常值

为了方便求解，引入拉格朗日乘数，把问题变成：
$$\min_{L,S}{{ \left\Vert {L} \right\Vert }_*+\lambda{ \left\Vert {S} \right\Vert }_1}+\mu{{ \left\Vert {N} \right\Vert }_F}$$
其中:噪声矩阵$N=M-L-S$  
建议的$\mu$取值为$\mu=\sqrt{2max(m,n)}\delta$，$\delta={ \left\Vert {N} \right\Vert }_F$ ，这样子的取值足够大能够排除数据噪声，也足够小不会导致原始矩阵被过分缩小

## 用pytorch实现spcp

In [ ]:
m,n = data.shape
m,n

spcp的代码

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt

LR = 0.01

L = torch.zeros([m,n],dtype=torch.float ,requires_grad=True)
S = torch.zeros([m,n],dtype=torch.float ,requires_grad=True)
M =  torch.tensor(data,dtype=torch.float ,requires_grad=False)

lambda1 = 1/(max(m,n)**0.5)
mu = 1/20 * torch.norm(M,p = 'nuc')

optimizer = torch.optim.Adam([L,S],lr = LR,betas = (0.9,0.99))    

t = 1
steps = []
losses = []
N = M-L-S
while torch.norm(N) > mu:
    steps.append(t)
    
    N = M-L-S
    loss = torch.norm(L,p = 'nuc')+lambda1 * torch.norm(S,p =1) + mu * torch.norm(N)
    losses.append(loss)
    
    
    optimizer.zero_grad()      #初始化梯度
    loss.backward()         #计算梯度
    optimizer.step()   
    '''
    if t%10 == 0:
        import IPython
        IPython.display.clear_output(wait=True)

        plt.plot(steps,losses,label = 'loss')
        plt.plot(steps,losses,label = 'loss')
        plt.show()
    '''
    t+=1
dataL = L.data.numpy()
dataS = S.data.numpy()
dataN = N.data.numpy()
data_ = dataL+dataS

In [ ]:


import numpy as np



import matplotlib.pyplot as plt


#设置图大小
size = 10

#让图转起来，每次绘图都清空掉上一次绘制结果
import IPython
IPython.display.clear_output(wait=True)

plt.figure(1,(8,8))

plt.scatter(data[:,0],data[:,1],label='origin data')
plt.scatter(data_[:,0],data_[:,1],label='restructured data')

vals,vecs = np.linalg.eig(np.cov(dataL.T))
vecs
i=0
#绘制两个主成分的方向
ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
ev = (ev+dataL.mean(0))
plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

i=1
ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
ev = (ev+dataL.mean(0))
plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))


#绘制一下原始数据和重构数据的连线
for j in range(len(data)):
    plt.plot([data[j,0],data_[j,0]],[data[j,1],data_[j,1]],c='black',linewidth = 0.5)


#画一下x轴y轴
plt.plot([-size,size],[0,0],c='black')
plt.plot([0,0],[-size,size],c='black')
plt.xlim(-12.5,12.5)
plt.ylim(-12.5,12.5)
plt.title('result of PCA')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt


def spcp(data):
    LR = 0.01

    L = torch.zeros([m,n],dtype=torch.float ,requires_grad=True)
    S = torch.zeros([m,n],dtype=torch.float ,requires_grad=True)
    M =  torch.tensor(data,dtype=torch.float ,requires_grad=False)
    N = M-L-S
    lambda1 = 1/(max(m,n)**0.5)
    mu = 1/20 * torch.norm(M,p = 'nuc')

    optimizer = torch.optim.Adam([L,S],lr = LR,betas = (0.9,0.99))    

    t = 1
    steps = []
    losses = []
    while torch.norm(N) > mu:
        steps.append(t)

        N = M-L-S
        loss = torch.norm(L,p = 'nuc')+lambda1 * torch.norm(S,p =1) + mu * torch.norm(N)
        losses.append(loss)


        optimizer.zero_grad()      #初始化梯度
        loss.backward()         #计算梯度
        optimizer.step()   
        '''
        if t%10 == 0:
            import IPython
            IPython.display.clear_output(wait=True)

            plt.plot(steps,losses,label = 'loss')
            plt.plot(steps,losses,label = 'loss')
            plt.show()
        '''
        t+=1
    dataL = L.data.numpy()
    dataS = S.data.numpy()
    dataN = N.data.numpy()
    return dataL,dataS,dataN
dataL,dataS,dataN = spcp(data)


In [ ]:
# 观察异常值对spcp的影响
for i in range(0,720,10):
    import matplotlib.pyplot as plt
    #设置图大小
    size = 10

    #让图转起来，每次绘图都清空掉上一次绘制结果
    import IPython
    IPython.display.clear_output(wait=True)

    plt.figure(1,(15,7))
    ax1=plt.subplot(121)
    ax2=plt.subplot(122)
    
    
    #造数据
    np.random.seed(0)
    data = np.random.uniform(1,10,(20,2))
    data[:,1:] = 0.5*data[:,0:1]+np.random.uniform(-2,2,(20,1))
    


    #如果我们在数据中加入异常值,定义一个角度放入异常值
    i = 2*i/360*np.pi
    data_outsider=[np.sin(i)*5*i/(2*np.pi),np.cos(i)*5*i/(2*np.pi)]
    data = np.vstack((data,data_outsider))
    
    
    #SPCP
    dataL,dataS,dataN = spcp(data)
    data_ = dataL+dataS
    

    import numpy as np





    
    plt.sca(ax1)
    plt.scatter(data[:,0],data[:,1],label='origin data')
    plt.scatter(data_[:,0],data_[:,1],label='restructured data')
    plt.scatter(data[-1,0],data[-1,1],label='outsider')
    
    vals,vecs = np.linalg.eig(np.cov(dataL.T))
    vecs
    i=0
    #绘制两个主成分的方向
    ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
    ev = (ev+dataL.mean(0))
    plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

    i=1
    ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
    ev = (ev+dataL.mean(0))
    plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

    #绘制一下原始数据和重构数据的连线
    for j in range(len(data)):
        plt.plot([data[j,0],data_[j,0]],[data[j,1],data_[j,1]],c='black',linewidth = 0.5)


    #画一下x轴y轴
    plt.plot([-size,size],[0,0],c='black')
    plt.plot([0,0],[-size,size],c='black')
    plt.xlim(-12.5,12.5)
    plt.ylim(-12.5,12.5)
    plt.title('result of spcp')
    plt.legend()
    
    
    #PCA
    plt.sca(ax2)
    #去中心化
    data_normal = data-data.mean(0)
    X = data_normal[:,0]
    Y = data_normal[:,1]
    
    #协方差矩阵
    C = np.cov(data_normal.T)
    
    #计算特征值和特征向量
    vals, vecs = np.linalg.eig(C)
    


    #数据在主成分1上的投影坐标是
    zcf1 = np.matmul(data_normal,vecs[:,0])
    
    #只用主成分1重构数据
    data_ = np.matmul(zcf1.reshape(len(data),1),vecs[:,0].reshape(1,2))+data.mean(0)

    plt.scatter(data[:,0],data[:,1],label='origin data')
    plt.scatter(data_[:,0],data_[:,1],label='restructured data')
    plt.scatter(data[-1,0],data[-1,1],label='outsider')
    
    i=0
    #绘制两个主成分的方向
    ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
    ev = (ev+data.mean(0))
    plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

    i=1
    ev = np.array([vecs[:,i]*-1,vecs[:,i]])*size
    ev = (ev+data.mean(0))
    plt.plot(ev[:,0],ev[:,1],label = 'eigen vector '+str(i+1))

    #绘制一下原始数据和重构数据的连线
    for j in range(len(data)):
        plt.plot([data[j,0],data_[j,0]],[data[j,1],data_[j,1]],c='black',linewidth = 0.5)


    #画一下x轴y轴
    plt.plot([-size,size],[0,0],c='black')
    plt.plot([0,0],[-size,size],c='black')
    plt.xlim(-12.5,12.5)
    plt.ylim(-12.5,12.5)
    plt.title('result of PCA')
    plt.legend()
    #
    
    plt.show()

In [ ]:
data.mean(0)